In [1]:
# Initialize Otter
import otter
grader = otter.Notebook("HW2_Gradescope.ipynb")

# Homework 2

## Instructions

1. Do not move or rename the included files. If you move or rename files or folders, the local and Gradescope autograders will not work.
1. Refer to files and variables exactly as they appear. Python is case-sensitive, so `df` and `DF` are different variables.
1. After you answer a question, you may run its public tests.
1. After you answer every question:
    1. Restart your kernel
    1. Run up to the last cell
    1. Save your notebook
    1. Run the last cell, which creates the .zip file for Gradescope in the same folder as this notebook
    1. Upload the .zip file to Gradescope
    1. Make sure your local autograder results match your Gradescope autograder results
1. ***The public tests ensure your answers are formatted correctly but do not ensure your answers are correct.***
1. Gradescope runs hidden tests in addition to the public tests but does not reveal public test results until after the due date. Your grade will be "~/100" until after the due date.

## Packages

In [2]:
import numpy as np
import pandas as pd
import datetime as dt
import warnings
warnings.filterwarnings('ignore')

## Questions

---

#### 1. Create a DataFrame named **fin_data** by reading in the columns labeled 'DATE', 'TICKER', 'PRC' and 'CFACPR' from the file entitled fin_data.csv. Set the column labeled 'DATE' as the index.

_Points:_ 9

In [3]:
# read in data
fin_data = pd.read_csv('fin_data.csv', usecols=['DATE', 'TICKER', 'PRC', 'CFACPR'])
fin_data['DATE'] = pd.to_datetime(fin_data['DATE'], format='%Y%m%d')
fin_data = fin_data.set_index('DATE')

fin_data

,TICKER,PRC,CFACPR
DATE,,,
2014-01-02,MSFT,37.16000,1
2014-01-03,MSFT,36.91000,1
2014-01-06,MSFT,36.13000,1
2014-01-07,MSFT,36.41000,1
2014-01-08,MSFT,35.76000,1
...,...,...,...
2018-12-24,UNH,232.94000,1
2018-12-26,UNH,243.35001,1
2018-12-27,UNH,245.97000,1


In [4]:
grader.check("q1")

q1 results: All test cases passed!

---

#### 2. Create a variable named **tickers** to store **an array** with the unique tickers for all the stocks in the sample, sorted in alphabetical order.

_Points:_ 9

In [5]:
tickers = np.array(sorted(fin_data['TICKER'].unique()))
tickers

array(['AAPL', 'AXP', 'BA', 'CAT', 'CSCO', 'CVX', 'DIS', 'GS', 'HD',
       'IBM', 'INTC', 'JNJ', 'JPM', 'KO', 'MCD', 'MMM', 'MRK', 'MSFT',
       'NKE', 'PFE', 'PG', 'TRV', 'UNH', 'UTX', 'V', 'VZ', 'WMT', 'XOM'],
      dtype='<U4')

In [6]:
grader.check("q2")

q2 results: All test cases passed!

---

#### 3. Create a new column (named 'RET') in the **fin_data** DataFrame to store the simple return computed as the change in the adjusted price. The adjusted price is the ratio of raw price ('PRC') and the cumulative factor to adjust prices ('CFACPR'). Subsequently, permanently drop the 'PRC' and 'CFACPR' columns from the DataFrame.

_Points:_ 11

In [7]:
grouped = fin_data.groupby('TICKER')
#fin_data['RET'] = 0
fin_data2 = pd.DataFrame()

for name, group in grouped:
    group['RET'] = (group['PRC'] / group['CFACPR']).div(group['PRC'].shift(1) / group['CFACPR'].shift(1)).sub(1)
    fin_data2 = fin_data2.append(group)
    
fin_data = fin_data2.copy()
fin_data.drop(columns=['PRC', 'CFACPR'], axis=1, inplace = True)

In [8]:
grader.check("q3")

q3 results: All test cases passed!

---

#### 4. Which stock had the lowest daily return during the sample period? In addition, what was the date and the observed return? Store the three values in three variables labeled **stock_min**,**date_min** and **ret_min**.

_Points:_ 9

In [9]:
# creating a new df as we want to keep the date indices in our original one but don't want the duplicate indices to get in the way
fin_data2 = fin_data.copy()
fin_data2 = fin_data2.reset_index()

# finding the minimum date, then the row associated with it, and indexing into it
min_by_date = fin_data2.groupby('DATE')['RET'].min()
date_min = min_by_date.idxmin()
min_row = fin_data2.loc[fin_data2['DATE'] == date_min].loc[fin_data2['RET'] == min_by_date[date_min]]

# getting the value at the 0th row position of the single row we extracted
stock_min = min_row.iloc[0]['TICKER']
date_min = min_row.iloc[0]['DATE']
ret_min = min_row.iloc[0]['RET']

In [10]:
grader.check("q4")

q4 results: All test cases passed!

---

#### 5. What is number of days that Goldman Sachs had a negative return during the sample period? Store the answer in a variable called **neg_days**.

_Points:_ 9

In [11]:
gs_data = fin_data.loc[fin_data['TICKER'] == 'GS']

# Count the number of days with a negative return
neg_days = (gs_data['RET'] < 0).sum()
neg_days

619

In [12]:
grader.check("q5")

q5 results: All test cases passed!

---

#### 6. Which stock had the highest monthly average daily return in 2018? In addition, what was the date and the observed return? Store the three values in three variables labeled **stock_max18**,**date_max18** and **ret_max18**.

_Points:_ 9

In [13]:
fin_data_2018 = fin_data.loc[fin_data.index.year == 2018]

# Compute the monthly average daily return for each stock
monthly_avg_returns = fin_data_2018.groupby([fin_data_2018.index.year, fin_data_2018.index.month, 'TICKER'])['RET'].mean()

# Find the highest monthly average daily return
ret_max18 = monthly_avg_returns.max()

stock_max18 = monthly_avg_returns.idxmax()[2]

# creating a date time object for the month is a bit more complex, we have to work backwards to get the day after the end
# of the month given by the 0th and 1st index
year = monthly_avg_returns.idxmax()[0]
month = monthly_avg_returns.idxmax()[1]

# now we get the date
next_month = pd.to_datetime(f'{year}-{month+1}-01')
date_max18 = next_month - pd.Timedelta(days=1)

In [14]:
grader.check("q6")

q6 results: All test cases passed!

---

#### 7. Create a DataFrame named **returns_2018** to store the 2018 monthly average daily returns for all the stocks in the sample. The DataFrame should have a column for each stock in the sample.

_Points:_ 9

In [15]:
# getting only 2018
fin_data_2018 = fin_data.loc[fin_data.index.year == 2018]

# grouping by year with a subindex of month in order to take the mean of the groupings
monthly_avg_returns = fin_data_2018.groupby([fin_data_2018.index.year, fin_data_2018.index.month, 'TICKER'])['RET'].mean()

# reshape the data
returns_2018 = monthly_avg_returns.unstack(level='TICKER')
returns_2018

TICKER         AAPL       AXP        BA       CAT      CSCO       CVX  \
DATE DATE                                                               
2018 1    -0.000460  0.000072  0.008985  0.001614  0.003924  0.000133   
     2     0.003508 -0.000690  0.001455 -0.002374  0.004270 -0.005692   
     3    -0.002713 -0.001994 -0.004525 -0.002010 -0.001897  0.000998   
     4    -0.000595  0.002927  0.001053 -0.000770  0.001677  0.004508   
     5     0.005692 -0.000156  0.002586  0.002401 -0.001577 -0.000214   
     6    -0.000414 -0.000111 -0.002199 -0.005276  0.000409  0.000893   
     7     0.001360  0.000808  0.002944  0.002933 -0.000733  0.000017   
     8     0.007915  0.002760 -0.001596 -0.001382  0.005350 -0.002720   
     9    -0.000341  0.000288  0.004362  0.004998  0.000980  0.001721   
     10   -0.001141 -0.001442 -0.001756 -0.009515 -0.002465 -0.003822   
     11   -0.009264  0.004338 -0.000881  0.005644  0.002372  0.003173   
     12   -0.006140 -0.008322 -0.003403 -0.003088 -0.004943 -0.004469   

TICKER          DIS        GS        HD       IBM  ...       NKE       PFE  \
DATE DATE                                          ...                       
2018 1     0.000593  0.002467  0.002822  0.003203  ...  0.004224  0.001167   
     2    -0.002546 -0.000710 -0.004843 -0.002425  ... -0.000677 -0.000864   
     3    -0.001210 -0.001874 -0.000976 -0.000636  ... -0.000342 -0.001019   
     4     0.000009 -0.002532  0.001860 -0.002474  ...  0.001540  0.001525   
     5    -0.000326 -0.002380  0.000474 -0.001129  ...  0.002282 -0.000809   
     6     0.002552 -0.001093  0.002167 -0.000514  ...  0.005278  0.000473   
     7     0.003864  0.003557  0.000602  0.001807  ... -0.001620  0.004613   
     8    -0.000566  0.000131  0.000732  0.000509  ...  0.002941  0.001745   
     9     0.002311 -0.003039  0.001703  0.001702  ...  0.001668  0.003159   
     10   -0.000634  0.000401 -0.006965 -0.011424  ... -0.005045 -0.000869   
     11    0.000337 -0.007667  0.001334  0.003656  ...  0.000180  0.003483   
     12   -0.002528 -0.006707 -0.002323 -0.004482  ... -0.000148 -0.002881   

TICKER           PG       TRV       UNH       UTX         V        VZ  \
DATE DATE                                                               
2018 1    -0.002911  0.004863  0.003510  0.003786  0.004122  0.001072   
     2    -0.004871 -0.003805 -0.002201 -0.001002 -0.000316 -0.006373   
     3     0.000497  0.000034 -0.002520 -0.003150 -0.001180  0.000110   
     4    -0.004285 -0.002485  0.004877 -0.002145  0.002928  0.001649   
     5     0.000561 -0.001026  0.001039  0.001774  0.001374 -0.001498   
     6     0.003131 -0.002304  0.000781  0.000108  0.000697  0.002658   
     7     0.001734  0.003019  0.001555  0.003998  0.001578  0.001282   
     8     0.001120  0.000511  0.002561 -0.001283  0.003134  0.002268   
     9     0.000213 -0.000709 -0.000432  0.003179  0.001193 -0.000910   
     10    0.002995 -0.001477 -0.000617 -0.004944 -0.003397  0.003019   
     11    0.003062  0.002074  0.003648 -0.000801  0.001489  0.002671   
     12   -0.001302 -0.004348 -0.006192 -0.006887 -0.003443 -0.003548   

TICKER          WMT       XOM  
DATE DATE                      
2018 1     0.003694  0.002080  
     2    -0.008465 -0.007231  
     3    -0.000468 -0.000638  
     4    -0.000203  0.002079  
     5    -0.003051  0.002090  
     6     0.001807  0.000924  
     7     0.001982 -0.000660  
     8     0.003330 -0.000680  
     9    -0.001061  0.003133  
     10    0.002942 -0.002730  
     11   -0.001186 -0.000004  
     12   -0.002311 -0.007857  

[12 rows x 28 columns]

In [16]:
grader.check("q7")

q7 results: All test cases passed!

---

#### 8. Create a DataFrame named **sp_data_2018** by reading in the columns labeled 'DATE' and 'SPRTRN' from the file entitled fin_data.csv. The column labeled 'SPRTRN' contains the daily return for the S&P500 index. Restrict the sample period to 2018. Set the column labeled 'DATE' as the index. Make sure the DataFrame does not have any duplicate observations. 

_Points:_ 9

In [17]:
# reading in data and looking at only 2018
sp_data_2018 = pd.read_csv('fin_data.csv', usecols=['DATE', 'SPRTRN'], parse_dates=['DATE'], index_col='DATE')
sp_data_2018 = sp_data_2018.loc[sp_data_2018.index.year == 2018]

# keeping only the first observation in the case of duplicate indicies
sp_data_2018 = sp_data_2018[~sp_data_2018.index.duplicated(keep='first')]
sp_data_2018

,SPRTRN
DATE,
2018-01-02,0.008303
2018-01-03,0.006399
2018-01-04,0.004029
2018-01-05,0.007034
2018-01-08,0.001662
...,...
2018-12-24,-0.027112
2018-12-26,0.049594
2018-12-27,0.008563


In [18]:
grader.check("q8")

q8 results: All test cases passed!

---

#### 9. Create a new DataFrame named **data_2018** by merging the DataFrame **returns_2018** with the monthly average daily return for the S&P500 index.

_Points:_ 9

In [19]:
# first, we need to group the data into the same indices as the other df in order to merge properly
monthly_sp = sp_data_2018.groupby([sp_data_2018.index.year, sp_data_2018.index.month])['SPRTRN'].mean()

data_2018 = returns_2018.merge(monthly_sp, left_index=True, right_index=True)
data_2018

AAPL       AXP        BA       CAT      CSCO       CVX  \
DATE DATE                                                               
2018 1    -0.000460  0.000072  0.008985  0.001614  0.003924  0.000133   
     2     0.003508 -0.000690  0.001455 -0.002374  0.004270 -0.005692   
     3    -0.002713 -0.001994 -0.004525 -0.002010 -0.001897  0.000998   
     4    -0.000595  0.002927  0.001053 -0.000770  0.001677  0.004508   
     5     0.005692 -0.000156  0.002586  0.002401 -0.001577 -0.000214   
     6    -0.000414 -0.000111 -0.002199 -0.005276  0.000409  0.000893   
     7     0.001360  0.000808  0.002944  0.002933 -0.000733  0.000017   
     8     0.007915  0.002760 -0.001596 -0.001382  0.005350 -0.002720   
     9    -0.000341  0.000288  0.004362  0.004998  0.000980  0.001721   
     10   -0.001141 -0.001442 -0.001756 -0.009515 -0.002465 -0.003822   
     11   -0.009264  0.004338 -0.000881  0.005644  0.002372  0.003173   
     12   -0.006140 -0.008322 -0.003403 -0.003088 -0.004943 -0.004469   

                DIS        GS        HD       IBM  ...       PFE        PG  \
DATE DATE                                          ...                       
2018 1     0.000593  0.002467  0.002822  0.003203  ...  0.001167 -0.002911   
     2    -0.002546 -0.000710 -0.004843 -0.002425  ... -0.000864 -0.004871   
     3    -0.001210 -0.001874 -0.000976 -0.000636  ... -0.001019  0.000497   
     4     0.000009 -0.002532  0.001860 -0.002474  ...  0.001525 -0.004285   
     5    -0.000326 -0.002380  0.000474 -0.001129  ... -0.000809  0.000561   
     6     0.002552 -0.001093  0.002167 -0.000514  ...  0.000473  0.003131   
     7     0.003864  0.003557  0.000602  0.001807  ...  0.004613  0.001734   
     8    -0.000566  0.000131  0.000732  0.000509  ...  0.001745  0.001120   
     9     0.002311 -0.003039  0.001703  0.001702  ...  0.003159  0.000213   
     10   -0.000634  0.000401 -0.006965 -0.011424  ... -0.000869  0.002995   
     11    0.000337 -0.007667  0.001334  0.003656  ...  0.003483  0.003062   
     12   -0.002528 -0.006707 -0.002323 -0.004482  ... -0.002881 -0.001302   

                TRV       UNH       UTX         V        VZ       WMT  \
DATE DATE                                                               
2018 1     0.004863  0.003510  0.003786  0.004122  0.001072  0.003694   
     2    -0.003805 -0.002201 -0.001002 -0.000316 -0.006373 -0.008465   
     3     0.000034 -0.002520 -0.003150 -0.001180  0.000110 -0.000468   
     4    -0.002485  0.004877 -0.002145  0.002928  0.001649 -0.000203   
     5    -0.001026  0.001039  0.001774  0.001374 -0.001498 -0.003051   
     6    -0.002304  0.000781  0.000108  0.000697  0.002658  0.001807   
     7     0.003019  0.001555  0.003998  0.001578  0.001282  0.001982   
     8     0.000511  0.002561 -0.001283  0.003134  0.002268  0.003330   
     9    -0.000709 -0.000432  0.003179  0.001193 -0.000910 -0.001061   
     10   -0.001477 -0.000617 -0.004944 -0.003397  0.003019  0.002942   
     11    0.002074  0.003648 -0.000801  0.001489  0.002671 -0.001186   
     12   -0.004348 -0.006192 -0.006887 -0.003443 -0.003548 -0.002311   

                XOM    SPRTRN  
DATE DATE                      
2018 1     0.002080  0.002621  
     2    -0.007231 -0.001948  
     3    -0.000638 -0.001220  
     4     0.002079  0.000186  
     5     0.002090  0.000993  
     6     0.000924  0.000245  
     7    -0.000660  0.001701  
     8    -0.000680  0.001306  
     9     0.003133  0.000232  
     10   -0.002730 -0.003027  
     11   -0.000004  0.000910  
     12   -0.007857 -0.004888  

[12 rows x 29 columns]

In [20]:
grader.check("q9")

q9 results: All test cases passed!

---

#### 10. Which stock had the highest correlation with the S&P500 in 2018, and what was the correlation magnitude? Store the values in two variables labeled **stock_corr_max** and **corr_max**.

_Points:_ 9

In [21]:
# creating a correlation matrix
corr_matrix = data_2018.corr()

# getting the stock and the correlation value
stock_corr_max = (corr_matrix.loc['SPRTRN'][:-1]).idxmax()
corr_max = corr_matrix.loc['SPRTRN', stock_corr_max]

In [22]:
grader.check("q10")

q10 results: All test cases passed!

---

#### 11. Use the returns_2018 DataFrame to create another DataFrame named **max_correlations** to store each stock's highest correlation during 2018 and the stock it occured with. Specifically, the DataFrame should have the following header:

_Points:_ 9

In [23]:
max_correlations = pd.DataFrame(columns = ['TICKER', 'CORR', 'TICKER_PAIR'])

# iterate through stocks
for stock in returns_2018.columns:
    corr_series = returns_2018.drop(columns=stock).corrwith(returns_2018[stock])

    # max correlation and the stock associated with it
    max_corr = corr_series.max()
    max_corr_stock = corr_series.idxmax()

    # add to dataframe
    max_correlations.loc[len(max_correlations.index)] = [stock, max_corr, max_corr_stock] 

max_correlations.set_index('TICKER', inplace = True)
max_correlations

,CORR,TICKER_PAIR
TICKER,,
AAPL,0.618578,GS
AXP,0.897287,UNH
BA,0.798559,UTX
CAT,0.835634,IBM
CSCO,0.748098,V
CVX,0.811421,XOM
DIS,0.795614,JNJ
GS,0.646325,JPM
HD,0.824701,IBM


In [24]:
grader.check("q11")

q11 results:
    q11 - 1 result:
        ✅ Test case passed

    q11 - 2 result:
        ❌ Test case failed
        Trying:
            (np.sum(max_correlations.reset_index().sort_values(by='TICKER')['TICKER_PAIR'].values==\
                   np.array(['CSCO', 'PFE', 'MSFT', 'MMM', 'AXP', 'HD', 'PFE', 'AAPL', 'NKE',
                   'HD', 'CAT', 'DIS', 'UTX', 'JNJ', 'VZ', 'IBM', 'MCD', 'JPM', 'V',
                   'DIS', 'KO', 'JPM', 'V', 'JPM', 'UNH', 'MRK', 'MRK', 'HD']))==28)|\
            (np.sum(max_correlations.reset_index().sort_values(by='TICKER_PAIR')['TICKER'].values==\
                   np.array(['GS', 'CSCO', 'INTC', 'AAPL', 'PFE', 'JNJ', 'IBM', 'XOM', 'CVX',
                   'MMM', 'KO', 'UTX', 'TRV', 'MSFT', 'PG', 'MRK', 'CAT', 'VZ', 'WMT',
                   'BA', 'HD', 'DIS', 'AXP', 'V', 'JPM', 'NKE', 'UNH', 'MCD']))==28)
        Expecting:
            True
        **********************************************************************
        Line 1, in q11 1
        Failed example:
            (np.sum(max_correlations.reset_index().sort_values(by='TICKER')['TICKER_PAIR'].values==\
                   np.array(['CSCO', 'PFE', 'MSFT', 'MMM', 'AXP', 'HD', 'PFE', 'AAPL', 'NKE',
                   'HD', 'CAT', 'DIS', 'UTX', 'JNJ', 'VZ', 'IBM', 'MCD', 'JPM', 'V',
                   'DIS', 'KO', 'JPM', 'V', 'JPM', 'UNH', 'MRK', 'MRK', 'HD']))==28)|\
            (np.sum(max_correlations.reset_index().sort_values(by='TICKER_PAIR')['TICKER'].values==\
                   np.array(['GS', 'CSCO', 'INTC', 'AAPL', 'PFE', 'JNJ', 'IBM', 'XOM', 'CVX',
                   'MMM', 'KO', 'UTX', 'TRV', 'MSFT', 'PG', 'MRK', 'CAT', 'VZ', 'WMT',
                   'BA', 'HD', 'DIS', 'AXP', 'V', 'JPM', 'NKE', 'UNH', 'MCD']))==28)
        Expected:
            True
        Got:
            False

    q11 - 3 result:
        ❌ Test case failed
        Trying:
            np.allclose(max_correlations['CORR'].sum(),20.68399012355372)
        Expecting:
            True
        **********************************************************************
        Line 1, in q11 2
        Failed example:
            np.allclose(max_correlations['CORR'].sum(),20.68399012355372)
        Expected:
            True
        Got:
            False

---

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

In [25]:
# Save your notebook first, then run this cell to export your submission.
grader.export(pdf=False, run_tests=True)

Running your submission against local test cases...



Your submission received the following results when run against available test cases:

    q1 results: All test cases passed!

    q2 results: All test cases passed!

    q3 results: All test cases passed!

    q4 results: All test cases passed!

    q5 results: All test cases passed!

    q6 results: All test cases passed!

    q7 results: All test cases passed!

    q8 results: All test cases passed!

    q9 results: All test cases passed!

    q10 results: All test cases passed!

    q11 results:
        q11 - 1 result:
            ✅ Test case passed

        q11 - 2 result:
            ❌ Test case failed
            Trying:
                (np.sum(max_correlations.reset_index().sort_values(by='TICKER')['TICKER_PAIR'].values==\
                       np.array(['CSCO', 'PFE', 'MSFT', 'MMM', 'AXP', 'HD', 'PFE', 'AAPL', 'NKE',
                       'HD', 'CAT', 'DIS', 'UTX', 'JNJ', 'VZ', 'IBM', 'MCD', 'JPM', 'V',
                 